<h1>GrEx1 Air Travel Customer Data</h1><br>
Eric Randall<br>
PREDICT 420: Summer 2017


In [6]:
import pandas as pd
import numpy as np
import pandas_profiling as pp

from pandas import Series,DataFrame

In [4]:
data2014 = pd.read_excel('grex1/sfo_custsat_2014.xlsx', sheetname='Sheet 1', header=0)
data2015 = pd.read_csv('grex1/sfo_custsat_2015.csv', header=0)
data2016 = pd.read_excel('grex1/sfo_custsat_2016.xlsx', header=0)

In [9]:
pp.ProfileReport(data2016)

Number of variables,97
Number of observations,3087
Total Missing (%),12.8%
Total size in memory,2.3 MiB
Average record size in memory,776.0 B
Numeric,66
Categorical,13
Date,1
Text (Unique),0
Rejected,17
Distinct count,3087


<b>1B:</b> Below we are renaming `Q12PRECHEKCRATE` to `Q12PRECHECKRATE` in the 2015 data, to fix the typo and make it consistent with the other datasets. Similarly, we are renaming `*RESPNUM` to `RESPNUM` for consistency.

In [3]:
data2014 = data2014.rename(columns={'Q18AGE': 'Q19AGE'})
data2014 = data2014.rename(columns={'Q19GENDER': 'Q20GENDER'})
data2014 = data2014.rename(columns={'Q20INCOME': 'Q21INCOME'})
data2014 = data2014.rename(columns={'Q21FLY': 'Q22FLY'})

data2015 = data2015.rename(columns={'Q12PRECHEKCRATE': 'Q12PRECHECKRATE'})
data2015 = data2015.rename(columns={'Q18AGE': 'Q19AGE'})
data2015 = data2015.rename(columns={'Q19GENDER': 'Q20GENDER'})
data2015 = data2015.rename(columns={'Q20INCOME': 'Q21INCOME'})
data2015 = data2015.rename(columns={'Q21FLY': 'Q22FLY'})

data2016 = data2015.rename(columns={'*RESPNUM': 'RESPNUM'})

In [23]:
# Add the year indicator to each DataFrame.  

data2014.insert(0, 'YEAR', 2014)
data2015.insert(0, 'YEAR', 2015)
data2016.insert(0, 'YEAR', 2016)

In [56]:
# As the RESPNUM repeat every year, combine the year and RESPNUM into one to keep it unique.  

data2014['RESPNUM'] = data2014.YEAR.apply(str).str.cat(data2014.RESPNUM.apply(str))
data2015['RESPNUM'] = data2015.YEAR.apply(str).str.cat(data2015.RESPNUM.apply(str))
data2016['RESPNUM'] = data2016.YEAR.apply(str).str.cat(data2016.RESPNUM.apply(str))

<b>1A:</b> Below we are removing all unneeded columns from the respective DataFrames. The remaining variable names are: <p>

* Respondent ID number: RESPNUM<br>
* Where the respondent lives relative to the airport: Q16LIVE<br>
* General assessments of aspects of the airport: Q7ART, Q7FOOD, Q7STORE, Q7SIGN, Q7WALKWAYS, Q7SCREENS, Q7INFODOWN, Q7INFOUP, Q7WIFI, Q7ROADS, Q7PARK, Q7AIRTRAIN, Q7LTPARKING, Q7RENTAL, Q7ALL<br>
* Assessments of cleanliness: Q9BOARDING, Q9AIRTRAIN, Q9RENTAL, Q9FOOD, Q9RESTROOM, Q9ALL<br>
* Impression of safety: Q10SAFE
* Impression of PreCheck: Q12PRECHECKRATE (As mentioned, in 2015 this is Q12PRECHEKCRATE due to a typo)
* Impression of ease of getting to the airport: Q13GETRATE
* Ease of finding way around the airport: Q14FIND
* Ease of getting through security: Q14PASSTHRU<br>
* Year of Survey: YEAR

In [22]:
keep_cols = ['RESPNUM', 'Q16LIVE', 'Q7ART', 'Q7FOOD', 'Q7STORE', 
             'Q7SIGN', 'Q7WALKWAYS', 'Q7SCREENS', 'Q7INFODOWN', 
             'Q7INFOUP', 'Q7WIFI', 'Q7ROADS', 'Q7PARK', 'Q7AIRTRAIN',
             'Q7LTPARKING', 'Q7RENTAL', 'Q7ALL', 'Q9BOARDING', 
             'Q9AIRTRAIN', 'Q9RENTAL', 'Q9FOOD', 'Q9RESTROOM', 
             'Q9ALL', 'Q10SAFE', 'Q12PRECHECKRATE', 'Q13GETRATE', 
             'Q14FIND', 'Q14PASSTHRU', 'YEAR'
            ]

data2014_trim = data2014[keep_cols]
data2015_trim = data2015[keep_cols]
data2016_trim = data2016[keep_cols]

In [51]:
# Merge the three DataFrames.  

air_survey = data2014_trim.append([data2015_trim, data2016_trim])

<b>1C:</b> Our resulting DataFrame has 29 columns (Including the index number) with 7 possible responses (0-6). Universally, the ratings are from 1-5 with 1 being the negative-most answer, 5 being the positive-most answer, 6 being Don't know/Not applicable, and 0 being left blank. The year variable is either 2014, 2015 or 2016.

Checking for NaN and Blank values, we find that Q7ALL (141 Blank entries), Q7FOOD (54 Blank entries) and Q12PRECHECKRATE (1 Blank entry) all need fixing. We convert them to 0, which is "No Response" in the survey.

In [52]:
air_survey = air_survey.replace(to_replace={'Q7FOOD': {'Blank': 0}})
air_survey = air_survey.replace(to_replace={'Q7ALL': {'Blank': 0}})
air_survey = air_survey.replace(to_replace={'Q16LIVE': {4: 0}})
air_survey = air_survey.replace(to_replace={'Q12PRECHECKRATE': {' ': 0}})

TypeError: Cannot compare types 'ndarray(dtype=float64)' and 'str'

In [53]:
air_survey.apply(pd.value_counts)

,Q10SAFE,Q12PRECHECKRATE,Q13GETRATE,Q14FIND,Q14PASSTHRU,Q16LIVE,Q7AIRTRAIN,Q7ALL,Q7ART,Q7FOOD,...,Q7WIFI,Q9AIRTRAIN,Q9ALL,Q9BOARDING,Q9FOOD,Q9RENTAL,Q9RESTROOM,RESPNUM,RESPNUM2,year
0.0,138.0,3642.0,943.0,234.0,484.0,290.0,505.0,357.0,238.0,164.0,...,276.0,388.0,179.0,114.0,319.0,458.0,186.0,NaN,NaN,NaN
1.0,20.0,52.0,116.0,54.0,79.0,2893.0,34.0,20.0,46.0,76.0,...,252.0,15.0,12.0,21.0,31.0,19.0,89.0,NaN,NaN,NaN
2.0,42.0,88.0,225.0,159.0,222.0,444.0,125.0,96.0,213.0,581.0,...,346.0,53.0,102.0,76.0,152.0,103.0,292.0,NaN,NaN,NaN
3.0,1014.0,375.0,1461.0,1061.0,1010.0,5107.0,798.0,1574.0,1760.0,2451.0,...,1119.0,607.0,1319.0,1001.0,1291.0,711.0,1718.0,NaN,NaN,NaN
4.0,2783.0,459.0,2076.0,2118.0,2001.0,NaN,1741.0,4522.0,2639.0,2865.0,...,1926.0,1745.0,3813.0,2993.0,2645.0,1291.0,3152.0,NaN,NaN,NaN
5.0,4737.0,1963.0,3478.0,5006.0,4155.0,NaN,1614.0,2085.0,2279.0,1286.0,...,2517.0,2199.0,3229.0,4463.0,2572.0,1494.0,2916.0,NaN,NaN,NaN
6.0,NaN,345.0,435.0,102.0,783.0,NaN,3917.0,80.0,1559.0,1311.0,...,2298.0,3727.0,80.0,66.0,1724.0,4658.0,381.0,NaN,NaN,NaN
201420141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2014201410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
20142014100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


<b>1D:</b> In the below cell we export our sheet to a CSV.

In [ ]:
air_survey.to_csv(header=True)

<b>2A:</b> The most common comments in 2015 were 999, 202 and 505.<P>
<i>999:</i> Good experience/keep up the good work/other positive comment (192 responses)<br>
<i>202:</i> Tables were not bussed/had to clear my own table to eat (142 responses)<br>
<i>505:</i> Add plugs/electrical outlets/places to charge devices (505 responses)<p>
The most common comments in 2016 were 0, 999, 202 and 103.<p>
<i>0:</i> No Response (1460 responses)<br>
<i>999:</i> Good experience/keep up the good work/other positive comment (193 responses)<br>
<i>202:</i> Tables were not bussed/had to clear my own table to eat (190 responses)<br>
<i>103:</i> Toilets splash water on you when flushed (139 responses)<br>

In [ ]:
pd.concat([data2015['Q8COM1'], data2015['Q8COM2'], data2015['Q8COM3']], axis=0, ignore_index=True).value_counts()

In [ ]:
pd.concat([data2016['Q8COM'], data2016['Q8COM2'], data2016['Q8COM3'], data2016['Q8COM4'], 
           data2016['Q8COM5']], axis=0, ignore_index=True).value_counts()

<b>3A:</b> Most respondents live in another region (#3 for `Q16LIVE`). The most frequent rating was a 4, which was true regardless of region.

In [54]:
pd.crosstab(air_survey["Q7ALL"], air_survey["Q16LIVE"], margins=True, normalize=True)

Q16LIVE,0,1,2,3,All
Q7ALL,,,,,
0,0.004580,0.010877,0.002748,0.022670,0.040875
1,0.000229,0.000458,0.000000,0.001603,0.002290
2,0.000458,0.003091,0.000687,0.006755,0.010992
3,0.006526,0.047057,0.008816,0.117815,0.180215
4,0.015113,0.184566,0.025074,0.292993,0.517747
5,0.005839,0.083581,0.012823,0.136478,0.238722
6,0.000458,0.001603,0.000687,0.006412,0.009160
All,0.033204,0.331234,0.050836,0.584726,1.000000


In [16]:
respondents = pd.read_csv('grex1/select_resps.csv', header=0)

In [ ]:
# As the RESPNUM repeat every year, combine the year and RESPNUM into one to keep it unique.  

respondents['RESPNUM'] = respondents.year.apply(str).str.cat(respondents.RESPNUM.apply(str))

In [65]:
# Pull in new data for the Respondents DataFrame.  

keep_cols_resp = ['RESPNUM', 'YEAR', 'DESTGEO', 'DESTMARK', 'Q2PURP1', 'Q2PURP2', 'Q2PURP3', 
                  'Q3PARK', 'Q4BAGS', 'Q4STORE', 'Q4FOOD', 'Q4WIFI', 'Q5TIMESFLOWN', 'Q5FIRSTTIME', 
                  'Q6LONGUSE','Q16LIVE', 'Q19AGE', 'Q20GENDER', 'Q21INCOME', 'Q22FLY', 'LANG', 
                  'Q23SJC', 'Q23OAK'
                 ]

data2014_resp = data2014[keep_cols_resp]
data2015_resp = data2015[keep_cols_resp]
data2016_resp = data2016[keep_cols_resp]

KeyError: "['YEAR' 'Q5FIRSTTIME' 'Q19AGE' 'Q20GENDER' 'Q21INCOME' 'Q22FLY' 'Q23SJC'\n 'Q23OAK'] not in index"

In [58]:
respondents = respondents.merge(air_survey, how='left', on='RESPNUM')

,RESPNUM,year_x,Q10SAFE,Q12PRECHECKRATE,Q13GETRATE,Q14FIND,Q14PASSTHRU,Q16LIVE,Q7AIRTRAIN,Q7ALL,...,Q7WALKWAYS,Q7WIFI,Q9AIRTRAIN,Q9ALL,Q9BOARDING,Q9FOOD,Q9RENTAL,Q9RESTROOM,RESPNUM2,year_y
0,20162401,2016,5.0,0.0,5.0,5.0,0.0,1.0,3.0,5.0,...,5.0,4.0,4.0,5.0,0.0,5.0,6.0,5.0,NaN,2016.0
1,20161559,2016,3.0,3.0,2.0,3.0,3.0,1.0,2.0,2.0,...,2.0,1.0,3.0,2.0,2.0,2.0,3.0,3.0,NaN,2016.0
2,20161561,2016,5.0,0.0,3.0,5.0,4.0,1.0,6.0,3.0,...,0.0,4.0,6.0,3.0,4.0,3.0,6.0,4.0,NaN,2016.0
3,20163045,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016837,2016,5.0,5.0,4.0,4.0,5.0,3.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,2016.0
5,20162880,2016,4.0,0.0,4.0,3.0,3.0,3.0,6.0,4.0,...,4.0,4.0,6.0,4.0,4.0,4.0,6.0,4.0,NaN,2016.0
6,20161535,2016,3.0,5.0,5.0,5.0,5.0,1.0,5.0,5.0,...,5.0,6.0,6.0,4.0,4.0,4.0,6.0,3.0,NaN,2016.0
7,20161296,2016,5.0,5.0,6.0,5.0,6.0,3.0,5.0,5.0,...,5.0,6.0,5.0,5.0,5.0,5.0,6.0,5.0,NaN,2016.0
8,2016409,2016,5.0,0.0,5.0,5.0,5.0,3.0,6.0,3.0,...,2.0,5.0,6.0,4.0,5.0,6.0,6.0,5.0,NaN,2016.0
9,2016120,2016,4.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,...,4.0,4.0,4.0,3.0,4.0,3.0,4.0,2.0,NaN,2016.0
